In [1]:
from Operations.CO_CompareMinAMI import CO_CompareMinAMI
from Operations.CO_AutoCorr import CO_AutoCorr
from Operations.CO_TranslateShape import CO_TranslateShape
from Operations.EN_ApEn import EN_ApEN
from Operations.DN_Moments import DN_Moments
from Operations.SY_SlidingWindow import SY_SlidingWindow
import numpy as np
import warnings

In [2]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")
ts4 = np.loadtxt("ts4.txt")

In [6]:
CO_TranslateShape(ts2, 'circle')

{'max': np.int64(3),
 'std': np.float64(0.24040380618965593),
 'mean': np.float64(2.9407630522088355),
 'npatmode': np.float64(0.9417670682730924),
 'mode': np.int64(3),
 'ones': np.float64(0.001004016064257028),
 'twos': np.float64(0.0572289156626506),
 'threes': np.float64(0.9417670682730924),
 'fours': np.float64(0.0),
 'fives': np.float64(0.0),
 'statav2_m': np.float64(0.017718851761347763),
 'statav2_s': np.float64(0.057191715948318844),
 'statav3_m': np.float64(0.04027548663053539),
 'statav3_s': np.float64(0.09234164438866023),
 'statav4_m': np.float64(0.03162298909764945),
 'statav4_s': np.float64(0.0731527606545579)}

In [132]:
np.std(7, ddof=1)

np.float64(nan)

In [23]:
np.arange(1+3, 100-3)

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
       55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
       72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88,
       89, 90, 91, 92, 93, 94, 95, 96])

In [82]:
def CO_TranslateShape(y, shape = 'circle', d = 2, howToMove = 'pts'):
    """
    """
    y = np.array(y, dtype=float)
    N = len(y)

    if y.ndim == 1:
        y = y.reshape(-1, 1)
    elif y.shape[1] > y.shape[0]:
        y = y.T

    # add a time index
    ty = np.column_stack((np.arange(1, N+1), y[:, 0])) # has increasing integers as time in the first column
    #-------------------------------------------------------------------------------
    # Generate the statistics on the number of points inside the shape as it is
    # translated across the time series
    #-------------------------------------------------------------------------------
    if howToMove == 'pts':

        if shape == 'circle':

            r = d # set radius
            w = int(np.floor(r))
            rnge = np.arange(1 + w, N - w + 1)
            NN = len(rnge) # number of admissible points
            np_counts = np.zeros(NN, dtype=int)

            for i in range(NN):
                idx = rnge[i]
                start = idx - w - 1
                end = idx + w
                win = ty[start:end, :]
                difwin = win - ty[idx - 1, :]
                squared_dists = np.sum(difwin**2, axis=1)
                np_counts[i] = np.sum(squared_dists <= r**2)

        elif shape == 'rectangle':

            w = d
            rnge = np.arange(1 + w, N - w + 1)
            NN = len(rnge)
            np_counts = np.zeros(NN, dtype=int)

            for i in range(NN):
                idx = rnge[i]
                start = (idx - w) - 1
                end = (idx + w)
                np_counts[i] = np.sum(
                    np.abs(y[start:end, 0]) <= np.abs(y[i, 0])
                )
        else:
            raise ValueError(f"Unknown shape {shape}. Choose either 'circle' or 'rectangle'")
    else:
        raise ValueError(f"Unknown setting for 'howToMove' input: '{howToMove}'. Only option is currently 'pts'.")

    # compute stats on number of hits inside the shape
    out = {}
    out["max"] = np.max(np_counts)
    out["std"] = np.std(np_counts, ddof=1)
    out["mean"] = np.mean(np_counts)
    
    # count the hits
    vals, hits = np.unique_counts(np_counts)
    max_val = np.argmax(hits)
    out["npatmode"] = hits[max_val]/NN
    out["mode"] = vals[max_val]

    count_types = ["ones", "twos", "threes", "fours", "fives", "sixes", "sevens", "eights", "nines", "tens", "elevens"]
    for i in range(1, 12):
        if 2*w + 1 >= i:
            out[f"{count_types[i-1]}"] = np.mean(np_counts == i)
    

    return out


In [90]:
def SY_SlidingWindow3(y : list, windowStat : str = 'mean', acrossWinStat : str = 'std', numSeg : int = 5, incMove : int = 2) -> dict:

    winLength = np.floor(len(y)/numSeg)
    if winLength == 0:
        warnings.warn(f"Time-series of length {len(y)} is too short for {numSeg} windows")
        return np.nan
    inc = np.floor(winLength/incMove) # increment to move at each step
    # if incrment rounded down to zero, prop it up 
    if inc == 0:
        inc = 1
    
    numSteps = int(np.floor((len(y)-winLength)/inc) + 1)
    qs = np.zeros(numSteps)
    
    # convert a step index (stepInd) to a range of indices corresponding to that window
    def get_window(stepInd: int):
        start_idx = (stepInd) * inc
        end_idx = (stepInd) * inc + winLength
        return np.arange(start_idx, end_idx).astype(int)

    if windowStat == 'mean':
        for i in range(numSteps):
            qs[i] = np.mean(y[get_window(i)])
    elif windowStat == 'std':
        for i in range(numSteps):
            qs[i] = np.std(y[get_window(i)], ddof=1)
    elif windowStat == 'ent':
        warnings.warn(f"{windowStat} not yet implemented")
    elif windowStat == 'apen':
        for i in range(numSteps):
            qs[i] = EN_ApEN(y[get_window(i)], 1, 0.2)
    elif windowStat == 'sampen':
        warnings.warn(f"{windowStat} not yet implemented")
    elif windowStat == 'mom3':
        for i in range(numSteps):
            qs[i] = DN_Moments(y[get_window(i)], 3)
    elif windowStat == 'mom4':
        for i in range(numSteps):
            qs[i] = DN_Moments(y[get_window(i)], 4)
    elif windowStat == 'mom5':
        for i in range(numSteps):
            qs[i] = DN_Moments(y[get_window(i)], 5)
    elif windowStat == 'AC1':
        for i in range(numSteps):
            qs[i] = CO_AutoCorr(y[get_window(i)], 1, 'Fourier')
    elif windowStat == 'lillie':
        warnings.warn(f"{windowStat} not yet implemented")
    else:
        raise ValueError(f"Unknown statistic '{windowStat}'")
    

    if acrossWinStat == 'std':
        out = np.std(qs, ddof=1)/np.std(y, ddof=1)
    
    return out
        


In [5]:
SY_SlidingWindow(ts1, 'AC1', 'apen')

np.float64(0.34431508471691297)